In [1]:
!pip install accelerate peft bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 40.0 MB/s eta 0:00:00


In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [3]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 131.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [4]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 35.5 MB/s eta 0:00:00


In [5]:
pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.1.1
    Uninstalling datasets-4.1.1:
      Successfully uninstalled datasets-4.1.1


In [6]:
from huggingface_hub import login
login()

In [7]:
from datasets import load_dataset
from collections import Counter

def load_bioinfer_kb():
    try:
        return load_dataset("bigbio/bioinfer", "bioinfer_bigbio_kb")
    except Exception as e:
        # Colab/ortamlarda script kısıtına takılırsan parquet fallback:
        if "Dataset scripts are no longer supported" in str(e):
            files = {
                "train": "hf://datasets/bigbio/bioinfer/bioinfer_bigbio_kb/train/*.parquet",
                "test":  "hf://datasets/bigbio/bioinfer/bioinfer_bigbio_kb/test/*.parquet",
            }
            return load_dataset("parquet", data_files=files)
        raise

raw = load_bioinfer_kb()

def relation_type_report(raw_ds):
    counts = Counter()
    for split in raw_ds.keys():
        for ex in raw_ds[split]:
            for r in ex.get("relations", []):
                t = (r.get("type") or "").upper()
                counts[t] += 1
    print("Unique relation types:", sorted(counts.keys()))
    print("Counts:", counts)
    # Sert doğrulama: yalnızca PPI bekliyoruz
    assert set(k for k in counts.keys() if k) == {"PPI"}, \
        f"Beklenmeyen relation type(lar) var: {set(counts.keys())}"

relation_type_report(raw)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

bioinfer.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/219k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/66.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/894 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/206 [00:00<?, ? examples/s]

Unique relation types: ['PPI']
Counts: Counter({'PPI': 2534})


In [8]:
import re, math, random
from typing import List, Dict, Any, Optional, Tuple, Set
from datasets import Dataset, DatasetDict

SENT_SPLIT = re.compile(r'(?<=[.!?])\s+')

def _first(x, default=""):
    if isinstance(x, list) and x:
        return x[0]
    return x if isinstance(x, str) else default

def _norm(s: str) -> str:
    return re.sub(r"\W+", " ", (s or "").lower()).strip()

def _sent_with_entities(full_text: str, h: str, t: str) -> str:
    sents = [s.strip() for s in SENT_SPLIT.split(full_text) if s.strip()]
    for s in sents:
        if h in s and t in s:
            return s
    return full_text.strip()

def build_pairs_from_bigbio_kb(example: Dict[str, Any],
                               negative_ratio: float = 1.0,
                               max_neg_per_doc: Optional[int] = None) -> List[Dict[str, str]]:
    # 1) Text
    texts = []
    for p in example.get("passages", []):
        for t in p.get("text", []):
            if isinstance(t, str) and t.strip():
                texts.append(t.strip())
    full_text = " ".join(texts).strip()

    # 2) Entities (id -> text)
    eid2txt = {}
    for e in example.get("entities", []):
        eid = e.get("id")
        et  = _first(e.get("text", []), "")
        if eid and isinstance(et, str) and et.strip():
            eid2txt[eid] = et.strip()
    if not eid2txt:
        return []

    # 3) Pozitif PPI çiftleri ve yüzey-form eşleri
    pos_ids: Set[Tuple[str, str]] = set()
    pos_surf: Set[Tuple[str, str]] = set()

    for r in example.get("relations", []):
        if (r.get("type") or "").upper() != "PPI":
            continue
        h_id, t_id = r.get("arg1_id"), r.get("arg2_id")
        if not h_id or not t_id:
            continue
        h_txt, t_txt = eid2txt.get(h_id, ""), eid2txt.get(t_id, "")
        if not h_txt or not t_txt:
            continue
        if _norm(h_txt) == _norm(t_txt):
            continue
        pos_ids.add((h_id, t_id)); pos_ids.add((t_id, h_id))
        pos_surf.add((_norm(h_txt), _norm(t_txt))); pos_surf.add((_norm(t_txt), _norm(h_txt)))

    out: List[Dict[str, str]] = []
    # Pozitif pair kayıtları (cümle seviyesine indir)
    seen_pos = set()
    for (h_id, t_id) in list(pos_ids):
        if (h_id, t_id) in seen_pos or h_id == t_id:
            continue
        seen_pos.add((h_id, t_id)); seen_pos.add((t_id, h_id))
        h_txt, t_txt = eid2txt.get(h_id, ""), eid2txt.get(t_id, "")
        if not h_txt or not t_txt:
            continue
        sent = _sent_with_entities(full_text, h_txt, t_txt)
        out.append({"sentence": sent, "h": h_txt, "t": t_txt, "label": "interaction"})

    # 4) Negatif adaylar: aynı cümlede geçen ama PPI olmayan çiftler
    eids = list(eid2txt.keys())
    cand_neg = []
    for i in range(len(eids)):
        for j in range(i+1, len(eids)):
            a, b = eids[i], eids[j]
            at, bt = eid2txt[a], eid2txt[b]
            if _norm(at) == _norm(bt):
                continue
            if (a, b) in pos_ids or (b, a) in pos_ids:
                continue
            if (_norm(at), _norm(bt)) in pos_surf or (_norm(bt), _norm(at)) in pos_surf:
                continue
            sent = _sent_with_entities(full_text, at, bt)
            if at in sent and bt in sent:
                cand_neg.append((a, b, sent))

    n_pos = sum(1 for x in out if x["label"] == "interaction")
    n_neg_target = int(math.ceil(negative_ratio * n_pos)) if n_pos > 0 else 0
    if max_neg_per_doc is not None:
        n_neg_target = min(n_neg_target, max_neg_per_doc)
    if n_neg_target > 0 and cand_neg:
        sampled = random.sample(cand_neg, k=min(n_neg_target, len(cand_neg)))
        for a, b, sent in sampled:
            out.append({"sentence": sent, "h": eid2txt[a], "t": eid2txt[b], "label": "no_interaction"})

    return out

def expand_split(split_ds, negative_ratio=1.0, max_neg_per_doc=None):
    rows = []
    for ex in split_ds:
        rows.extend(build_pairs_from_bigbio_kb(ex, negative_ratio, max_neg_per_doc))
    return Dataset.from_list(rows)

NEGATIVE_RATIO = 1.0
MAX_NEG_PER_DOC = None

train_pairs = expand_split(raw["train"], NEGATIVE_RATIO, MAX_NEG_PER_DOC)
val_base    = raw["test"] if "test" in raw else raw["validation"]
val_pairs   = expand_split(val_base, NEGATIVE_RATIO, MAX_NEG_PER_DOC)

ds_pairs = DatasetDict({"train": train_pairs, "val": val_pairs})
print(ds_pairs)
print("Sample:", ds_pairs["train"][0])


DatasetDict({
    train: Dataset({
        features: ['sentence', 'h', 't', 'label'],
        num_rows: 3208
    })
    val: Dataset({
        features: ['sentence', 'h', 't', 'label'],
        num_rows: 645
    })
})
Sample: {'sentence': 'alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex.', 'h': 'beta-catenin', 't': 'T-cell factor', 'label': 'interaction'}


In [11]:
SYSTEM_PROMPT = "You are a precise biomedical relation extraction assistant."

def to_messages(ex):
    sent, h, t = ex["sentence"], ex["h"], ex["t"]
    def mark_once(s, sub, L, R):
        if sub and sub in s:
            i = s.find(sub)
            return s[:i] + L + sub + R + s[i+len(sub):]
        return s
    marked = mark_once(sent, h, "[E1]", "[/E1]")
    marked = mark_once(marked, t, "[E2]", "[/E2]")
    user = (
        "Decide if the two marked entities interact (PPI: bind/complex/association). "
        "Only physical PPI counts as interaction.\n"
        f"Sentence: {marked}\n"
        f"Entity 1: {h}\n"
        f"Entity 2: {t}\n"
        "Answer with exactly one label: interaction or no_interaction. Output nothing else."
    )
    return [
        {"role":"system","content": SYSTEM_PROMPT},
        {"role":"user","content": user},
    ]

def render_prompt(tokenizer, ex):
    msgs = to_messages(ex)
    if hasattr(tokenizer, "apply_chat_template") and tokenizer.chat_template is not None:
        return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    return "SYSTEM: "+msgs[0]["content"]+"\n\nUSER: "+msgs[1]["content"]+"\n\nASSISTANT:"

import torch

@torch.no_grad()
def generate_label(model, tokenizer, ex, max_new_tokens=6, do_sample=False):
    prompt = render_prompt(tokenizer, ex)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
    )
    gen = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True).strip().lower()
    # basit post-proc
    if "interaction" in gen and "no_interaction" not in gen:
        return "interaction"
    if "no_interaction" in gen:
        return "no_interaction"
    return "no_interaction"  # temkinli fallback


In [14]:
class AssistantOnlyDataset(torch.utils.data.Dataset):
    def __init__(self, hf_split, tokenizer, max_len: int):
        self.data = hf_split
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.data)

    def __getitem__(self, i):
        ex = self.data[i]
        prompt, target = render_prompt_and_target(self.tok, ex)
        full = prompt + target
        tok_full = self.tok(full, max_length=self.max_len, truncation=True, padding=False)
        tok_prompt = self.tok(prompt, add_special_tokens=False)

        input_ids = tok_full["input_ids"]
        attention_mask = tok_full["attention_mask"]
        cutoff = len(tok_prompt["input_ids"])

        labels = [-100] * len(input_ids)
        for j in range(cutoff, len(input_ids)):
            labels[j] = input_ids[j]

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
        }

def make_splits(tokenizer, ds: DatasetDict, max_len: int):
    return (
        AssistantOnlyDataset(ds["train"], tokenizer, max_len),
        AssistantOnlyDataset(ds["val"], tokenizer, max_len),
    )

In [15]:

import os, math, random
from typing import List, Dict, Any, Optional, Tuple, Set

import numpy as np
import torch

from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# Use an Instruct chat model (requires HF access for Llama 3)
BASE_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"  # change if needed
OUTPUT_DIR = "./outputs_bioinfer_llama3_gen"

MAX_SEQ_LEN = 384
NEGATIVE_RATIO = 1.0
MAX_NEG_PER_DOC = None

BATCH_TRAIN = 2
BATCH_EVAL = 2
EPOCHS = 1
LEARNING_RATE = 1e-4

LABELS = ["no_interaction", "interaction"]


tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

# --- Optional LoRA / QLoRA ---
# model = prepare_model_for_kbit_training(model)  # enable if loading in 4/8bit
# lora_cfg = LoraConfig(
#     r=8, lora_alpha=32, lora_dropout=0.05, bias="none",
#     target_modules=None
# )
# model = get_peft_model(model, lora_cfg)

train_set, val_set = make_splits(tokenizer, ds, MAX_SEQ_LEN)
print("Train/Val sizes:", len(train_set), len(val_set))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

NameError: name 'ds' is not defined

In [10]:
ex = {
  "sentence": "alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex.",
  "h": "T-cell factor",
  "t": "alpha-catenin",
  "label": "interaction"
}
print("PRED:", generate_label(model, tokenizer, ex), "| GOLD:", ex["label"])


NameError: name 'model' is not defined

In [ ]:
import random
def sample_balanced(ds_split, per_label=3, seed=42):
    random.seed(seed)
    buckets = {"interaction": [], "no_interaction": []}
    idxs = list(range(len(ds_split)))
    random.shuffle(idxs)
    for i in idxs:
        lab = ds_split[i]["label"]
        if lab in buckets and len(buckets[lab]) < per_label:
            buckets[lab].append(i)
        if all(len(v) >= per_label for v in buckets.values()):
            break
    return buckets["interaction"] + buckets["no_interaction"]

idxs = sample_balanced(ds_pairs["val"], per_label=3)
for i in idxs:
    ex = ds_pairs["val"][i]
    pred = generate_label(model, tokenizer, ex)
    preview = ex["sentence"][:140] + ("..." if len(ex["sentence"])>140 else "")
    print(f"- SENT: {preview}\n  HEAD: {ex['h']} | TAIL: {ex['t']}\n  PRED: {pred} | GOLD: {ex['label']}\n")

In [16]:
# ==== BOOTSTRAP: model + tokenizer + BioInfer(PPI) ikili veri + jeneratif inference ====
import re, math, random, os
from typing import List, Dict, Any, Optional, Tuple, Set
import torch
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForCausalLM

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------------------------
# 1) BioInfer (KB) yükle (script engeli varsa parquet fallback)
# ---------------------------
def load_bioinfer_kb():
    try:
        return load_dataset("bigbio/bioinfer", "bioinfer_bigbio_kb")
    except Exception as e:
        if "Dataset scripts are no longer supported" not in str(e):
            raise
        files = {
            "train": "hf://datasets/bigbio/bioinfer/bioinfer_bigbio_kb/train/*.parquet",
            "test":  "hf://datasets/bigbio/bioinfer/bioinfer_bigbio_kb/test/*.parquet",
        }
        return load_dataset("parquet", data_files=files)

raw = load_bioinfer_kb()

# Tip doğrulama: pratikte PPI olmalı
from collections import Counter
ctr = Counter()
for split in raw.keys():
    for ex in raw[split]:
        for r in ex.get("relations", []):
            ctr[(r.get("type") or "").upper()] += 1
print("Relation types:", dict(ctr))  # beklenen: {'PPI': N}

# ---------------------------
# 2) PPI ikili pair seti oluştur (pozitif=PPI, negatif=aynı cümlede geçen ama PPI olmayan)
# ---------------------------
SENT_SPLIT = re.compile(r'(?<=[.!?])\s+')

def _first(x, default=""):
    if isinstance(x, list) and x:
        return x[0]
    return x if isinstance(x, str) else default

def _norm(s: str) -> str:
    return re.sub(r"\W+", " ", (s or "").lower()).strip()

def _sent_with_entities(full_text: str, h: str, t: str) -> str:
    sents = [s.strip() for s in SENT_SPLIT.split(full_text) if s.strip()]
    for s in sents:
        if h in s and t in s:
            return s
    return full_text.strip()

def build_pairs_from_bigbio_kb(example: Dict[str, Any],
                               negative_ratio: float = 1.0,
                               max_neg_per_doc: Optional[int] = None) -> List[Dict[str, str]]:
    # metin
    texts = []
    for p in example.get("passages", []):
        for t in p.get("text", []):
            if isinstance(t, str) and t.strip():
                texts.append(t.strip())
    full_text = " ".join(texts).strip()

    # entity id -> text
    eid2txt = {}
    for e in example.get("entities", []):
        eid = e.get("id")
        et  = _first(e.get("text", []), "")
        if eid and isinstance(et, str) and et.strip():
            eid2txt[eid] = et.strip()
    if not eid2txt:
        return []

    # pozitif PPI
    pos_ids: Set[Tuple[str, str]] = set()
    pos_surf: Set[Tuple[str, str]] = set()
    for r in example.get("relations", []):
        if (r.get("type") or "").upper() != "PPI":
            continue
        h_id, t_id = r.get("arg1_id"), r.get("arg2_id")
        if not h_id or not t_id:
            continue
        h_txt, t_txt = eid2txt.get(h_id, ""), eid2txt.get(t_id, "")
        if not h_txt or not t_txt:
            continue
        if _norm(h_txt) == _norm(t_txt):  # self/surface-equal
            continue
        pos_ids.add((h_id, t_id)); pos_ids.add((t_id, h_id))
        pos_surf.add((_norm(h_txt), _norm(t_txt))); pos_surf.add((_norm(t_txt), _norm(h_txt)))

    out: List[Dict[str, str]] = []
    seen_pos = set()
    for (h_id, t_id) in list(pos_ids):
        if (h_id, t_id) in seen_pos or h_id == t_id:
            continue
        seen_pos.add((h_id, t_id)); seen_pos.add((t_id, h_id))
        h_txt, t_txt = eid2txt.get(h_id, ""), eid2txt.get(t_id, "")
        if not h_txt or not t_txt:
            continue
        sent = _sent_with_entities(full_text, h_txt, t_txt)
        out.append({"sentence": sent, "h": h_txt, "t": t_txt, "label": "interaction"})

    # negatif adaylar: aynı cümlede geçen ama PPI olmayan çiftler
    eids = list(eid2txt.keys())
    cand_neg = []
    for i in range(len(eids)):
        for j in range(i+1, len(eids)):
            a, b = eids[i], eids[j]
            at, bt = eid2txt[a], eid2txt[b]
            if _norm(at) == _norm(bt):
                continue
            if (a, b) in pos_ids or (b, a) in pos_ids:
                continue
            if (_norm(at), _norm(bt)) in pos_surf or (_norm(bt), _norm(at)) in pos_surf:
                continue
            sent = _sent_with_entities(full_text, at, bt)
            if at in sent and bt in sent:
                cand_neg.append((a, b, sent))

    n_pos = sum(1 for x in out if x["label"] == "interaction")
    n_neg_target = int(math.ceil(1.0 * n_pos)) if n_pos > 0 else 0
    if max_neg_per_doc is not None:
        n_neg_target = min(n_neg_target, max_neg_per_doc)
    if n_neg_target > 0 and cand_neg:
        sampled = random.sample(cand_neg, k=min(n_neg_target, len(cand_neg)))
        for a, b, sent in sampled:
            out.append({"sentence": sent, "h": eid2txt[a], "t": eid2txt[b], "label": "no_interaction"})

    return out

def expand_split(split_ds, negative_ratio=1.0, max_neg_per_doc=None):
    rows = []
    for ex in split_ds:
        rows.extend(build_pairs_from_bigbio_kb(ex, negative_ratio, max_neg_per_doc))
    return Dataset.from_list(rows)

NEGATIVE_RATIO = 1.0
MAX_NEG_PER_DOC = None
train_pairs = expand_split(raw["train"], NEGATIVE_RATIO, MAX_NEG_PER_DOC)
val_base    = raw["test"] if "test" in raw else raw["validation"]
val_pairs   = expand_split(val_base, NEGATIVE_RATIO, MAX_NEG_PER_DOC)
ds_pairs = DatasetDict({"train": train_pairs, "val": val_pairs})
print(ds_pairs)
print("Sample:", ds_pairs["train"][0])

# ---------------------------
# 3) Model & tokenizer (Llama 3 Instruct; düşerse küçük fallback)
# ---------------------------
MODEL_CANDIDATES = [
    "meta-llama/Meta-Llama-3-8B-Instruct",      # erişimin varsa
    "meta-llama/Llama-3.2-1B-Instruct",         # küçük Llama (daha erişilebilir)
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",       # serbest alternatif
    "HuggingFaceTB/SmolLM2-360M-Instruct",      # çok küçük, serbest
]
model = None; tokenizer = None; chosen = None
for mid in MODEL_CANDIDATES:
    try:
        tok = AutoTokenizer.from_pretrained(mid, use_fast=True)
        if tok.pad_token is None:
            tok.pad_token = tok.eos_token
        mdl = AutoModelForCausalLM.from_pretrained(
            mid, dtype=(torch.float16 if torch.cuda.is_available() else torch.float32)
        )
        tokenizer, model, chosen = tok, mdl, mid
        break
    except Exception as e:
        print(f"[WARN] Model yüklenemedi: {mid} -> {e}")

assert model is not None and tokenizer is not None, "Hiçbir model yüklenemedi. HF token gerekebilir."

print("Using base model:", chosen)

# ---------------------------
# 4) Jeneratif prompt + inference
# ---------------------------
SYSTEM_PROMPT = "You are a precise biomedical relation extraction assistant."

def to_messages(ex):
    sent, h, t = ex["sentence"], ex["h"], ex["t"]
    def mark_once(s, sub, L, R):
        if sub and sub in s:
            i = s.find(sub); return s[:i] + L + sub + R + s[i+len(sub):]
        return s
    marked = mark_once(sent, h, "[E1]", "[/E1]")
    marked = mark_once(marked, t, "[E2]", "[/E2]")
    user = (
        "Decide if the two marked entities physically interact (PPI: bind/complex/association). "
        "Regulatory effects without binding count as no_interaction.\n"
        f"Sentence: {marked}\n"
        f"Entity 1: {h}\n"
        f"Entity 2: {t}\n"
        "Answer with exactly one label: interaction or no_interaction. Output nothing else."
    )
    return [{"role":"system","content": SYSTEM_PROMPT},
            {"role":"user","content": user}]

def render_prompt(tokenizer, ex):
    msgs = to_messages(ex)
    if hasattr(tokenizer, "apply_chat_template") and tokenizer.chat_template is not None:
        return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    return "SYSTEM: "+msgs[0]["content"]+"\n\nUSER: "+msgs[1]["content"]+"\n\nASSISTANT:"

@torch.no_grad()
def generate_label(model, tokenizer, ex, max_new_tokens=6, do_sample=False):
    prompt = render_prompt(tokenizer, ex)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
    )
    gen = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True).strip().lower()
    if "no_interaction" in gen:
        return "no_interaction"
    if "interaction" in gen:
        return "interaction"
    return "no_interaction"

# ---------------------------
# 5) Hızlı demo: dengeli örneklem + metrik
# ---------------------------
def sample_balanced(ds_split, per_label=3, seed=42):
    random.seed(seed)
    buckets = {"interaction": [], "no_interaction": []}
    idxs = list(range(len(ds_split)))
    random.shuffle(idxs)
    for i in idxs:
        lab = ds_split[i]["label"]
        if lab in buckets and len(buckets[lab]) < per_label:
            buckets[lab].append(i)
        if all(len(v) >= per_label for v in buckets.values()):
            break
    return buckets["interaction"] + buckets["no_interaction"]

idxs = sample_balanced(ds_pairs["val"], per_label=3)
preds, golds = [], []
print("\n=== Diverse predictions on VAL ===")
for i in idxs:
    ex = ds_pairs["val"][i]
    pred = generate_label(model, tokenizer, ex)
    gold = ex["label"]
    preds.append(pred); golds.append(gold)
    preview = ex["sentence"][:160] + ("..." if len(ex["sentence"])>160 else "")
    print(f"- SENT: {preview}")
    print(f"  HEAD: {ex['h']} | TAIL: {ex['t']}")
    print(f"  PRED: {pred} | GOLD: {gold}\n")

from sklearn.metrics import classification_report, confusion_matrix
print("Macro-F1 on sample:",
      classification_report(golds, preds, labels=["no_interaction","interaction"], output_dict=True)["macro avg"]["f1-score"])
print("Confusion [rows=true, cols=pred]:",
      confusion_matrix(golds, preds, labels=["no_interaction","interaction"]).tolist())


Relation types: {'PPI': 2534}
DatasetDict({
    train: Dataset({
        features: ['sentence', 'h', 't', 'label'],
        num_rows: 3208
    })
    val: Dataset({
        features: ['sentence', 'h', 't', 'label'],
        num_rows: 645
    })
})
Sample: {'sentence': 'alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex.', 'h': 'beta-catenin', 't': 'T-cell factor', 'label': 'interaction'}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Using base model: meta-llama/Meta-Llama-3-8B-Instruct

=== Diverse predictions on VAL ===
- SENT: Binding to human dipeptidyl peptidase IV by adenosine deaminase and antibodies that inhibit ligand binding involves overlapping, discontinuous sites on a predic...
  HEAD: dipeptidyl peptidase IV | TAIL: adenosine deaminase
  PRED: interaction | GOLD: interaction

- SENT: In investigating the mechanism by which pRb induces senescence, we have found that pRb causes a posttranscriptional accumulation of the cyclin-dependent kinase ...
  HEAD: KIP1 | TAIL: cyclin E
  PRED: interaction | GOLD: interaction

- SENT: Genetic analysis suggested that this phenotype was due to msh6-F337A sequestering MSH2 and preventing it from interacting with MSH3 and MSH6.
  HEAD: MSH3 | TAIL: msh6-F337A
  PRED: interaction | GOLD: interaction

- SENT: Although both receptors are involved in M21 and H2981 cell adhesion to vitronectin, only a alpha v beta 3 can be detected in focal contacts, colocalizing with v...